<a href="https://colab.research.google.com/github/dmbrundage88/genomic_datascience/blob/main/Algo_for_DNA_Sequencing_Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#HelperFuntion
#Boyer Moore
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

#Naive Exact Matching Algo
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences
#Find Reverse Compliment
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t
#Read FastA format
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [28]:
#get module for Boyer Moore preprocessing and fasta data
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

--2020-10-14 16:55:37--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 54.230.85.146, 54.230.85.88, 54.230.85.98, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|54.230.85.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: ‘bm_preproc.py.1’

bm_preproc.py.1     100%[===================>]   9.18K  --.-KB/s    in 0s      

2020-10-14 16:55:38 (182 MB/s) - ‘bm_preproc.py.1’ saved [9400/9400]

--2020-10-14 16:55:38--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 54.230.85.98, 54.230.85.8, 54.230.85.146, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|54.230.85.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/o

In [30]:
from bm_preproc import BoyerMoore
import kmer_index

In [26]:
# implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return 
# (a) the number of character comparisons performed and 
# (b) the number of alignments tried. 
# Roughly speaking, these measure how much work the two different algorithms are doing.
genome = readGenome('chr1.GRCh38.excerpt.fasta')
#get counts for naive
def naive(p, t):
    occurrences=[]
    alignments = 0
    chars = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        alignments+=1
        match = True
        for j in range(len(p)):  # loop over characters
            chars+=1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return alignments, chars, occurrences
p='GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t=genome
alignments, chars, occurrences = naive(p, genome)
print(f'Alignments: {alignments} Characters:{chars}')

Alignments: 799954 Characters:984143


In [27]:
#get counts for Boyer-Moore
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    alignments = 0
    chars = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            chars+=1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
        alignments+=1
    return alignments, chars, occurrences

p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = genome
index = 'AGCT'
p_bm = BoyerMoore(p, index)   
alignments, chars, occurrences = boyer_moore(p,p_bm, t)
print(f'Alignments: {alignments} Characters:{chars}')

Alignments: 127974 Characters:165191


In [45]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits
kmerIndex = kmer_index.Index(genome, 2)
hits = kmerIndex.query('GGCGCGGTGGCTCACGCCTGTAAT')
print(len(hits))

32162


In [58]:
def approximate_match(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    p_idx = Index(t, segment_length)
    idx_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = p_idx.query(p[start:end])
        
        # Extend matching segments to see if whole p matches
        for m in matches:
            idx_hits += 1
            if m < start or m-start+len(p) > len(t):
                continue
            
            mismatches = 0
            
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), idx_hits

approximate_match('GGCGCGGTGGCTCACGCCTGTAAT', genome, 2)

([84641,
  160162,
  724927,
  273669,
  147558,
  364263,
  421221,
  681737,
  717706,
  465647,
  429299,
  747359,
  657496,
  160729,
  56922,
  635931,
  191452,
  551134,
  262042],
 90)

In [43]:
#check KMER
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mismatch=0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
              mismatch+=1
            if mismatch >2:
                match= False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

occur = naive_2mm("GGCGCGGTGGCTCACGCCTGTAAT", genome)
print(occur)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


In [47]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

ind = SubseqIndex('GGCGCGGTGGCTCACGCCTGTAAT', 8, 3)
print(ind.index)

[('CGGTCCTT', 2), ('GCTCACGA', 1), ('GGGGCGTA', 0)]
